In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# THIS DATASET HAS TAKEN CARE OF OUTLIERS AND MISSING VALUES 
dataset=pd.read_csv('/kaggle/input/outlineddataset/outliners.csv')

In [ ]:
dataset.shape

In [ ]:
# MIN MAX SCALING 
from sklearn.preprocessing import MinMaxScaler
def minmax_scaling(cur_col):
    min_val = np.min(cur_col)
    max_val = np.max(cur_col)
    scaled_col = (cur_col - min_val) / (max_val - min_val)
    return scaled_col
for index in range(2, dataset.shape[1]):
    cur_col = dataset.iloc[:, index]
    scaled_col = minmax_scaling(cur_col)
    dataset.iloc[:, index] = scaled_col

In [ ]:
# EXTRACTING 9999 VALUES FROM THE DATASET AS THE DATASET IS IMBALANCED AND LARGE
start_row = 2712
end_row = 12710
new_dataset = dataset.iloc[start_row-1:end_row]
#new_dataset.to_csv('new_dataset.csv', index=False)
new_dataset_flag=new_dataset.iloc[0:4998]
for i in range(0, new_dataset_flag.shape[0]):
    new_dataset_flag.iloc[i, 1] = 1    
new_dataset_flag    


In [ ]:
# CREATING CYNTHETIC THEFT ATTACKS AS THE VALUES OF THEFT USERS GIVEN IN THE DATASET IS TOO LESS 

import random
# Applying attacks to benign ranges
dataset_t1=new_dataset.iloc[901:1584]
dataset_t2=new_dataset.iloc[1584:2267]
dataset_t3=new_dataset.iloc[2267:2950]
dataset_t4=new_dataset.iloc[2950:3633]
dataset_t5=new_dataset.iloc[3633:4316]
dataset_t6=new_dataset.iloc[4316:4999]

#TA1
for i in range(0, dataset_t1.shape[0]):
    n = random.uniform(0.1, 0.9)
    for j in range(2, 1036):
        dataset_t1.iloc[i, j] = n*dataset_t1.iloc[i,j]


In [ ]:
#TA2        
for i in range(0, dataset_t2.shape[0]):
    for j in range(2, 1036):
        n = random.uniform(0.1,1)
        dataset_t2.iloc[i, j] = n*dataset_t2.iloc[i,j]


In [ ]:
#TA3       
for i in range(0, dataset_t3.shape[0]):
    n = random.randint(0,1)
    for j in range(2, 1036):
        dataset_t3.iloc[i, j] = n*dataset_t3.iloc[i,j]


In [ ]:
        
#TA4        
for i in range(0, dataset_t4.shape[0]):
    n = random.uniform(0.1,1)
    row_data=dataset_t4.iloc[i]
    mean_value = row_data[2:].mean()
    for j in range(2, 1036):
        dataset_t4.iloc[i, j] = n*mean_value



In [ ]:
        
#TA5          
for i in range(0, dataset_t5.shape[0]):
    row_data=dataset_t5.iloc[i]
    mean_value = row_data[2:].mean()
    for j in range(2, 1036):
        dataset_t5.iloc[i, j] = mean_value        


In [ ]:
        
 #TA6         
for i in range(0, dataset_t6.shape[0]):
    start_index = 2  
    dataset_t6.iloc[:, start_index:] = dataset_t6.iloc[:, start_index:].apply(lambda row: row[::-1], axis=1)
               

In [ ]:
# SEPARATING DEPENDENT AND INDEPENDENT VARIABLES
x = new_dataset.iloc[:, 2:].values
y = new_dataset.iloc[:, 1].values
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# CNN GRU Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, GRU, Dense

sequence_length = x.shape[1]  
num_classes = len(np.unique(y))  

model = Sequential()

# Convolutional layer
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(sequence_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

# GRU layer
model.add(GRU(units=64))

# Dense layer for output
model.add(Dense(units=num_classes, activation='softmax'))

# Compile
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


In [ ]:
# Train the model
history=model.fit(x_train, y_train, epochs=100, batch_size=32)



In [ ]:
# Test The Model
loss = model.evaluate(x_test, y_test) 
print("Test Loss:", loss)

In [ ]:
y_pred =model.predict(x_test)
y_pred_classes = np.round(y_pred).astype(int)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
import numpy as np

y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# accuracy
accuracy = accuracy_score(y_test, y_pred_classes)
print("Accuracy:", accuracy)

loss, _ = model.evaluate(x_test, y_test, verbose=0)
print("Test Loss:", loss)

# precision
precision = precision_score(y_test, y_pred_classes, average='weighted')
print("Precision:", precision)

# confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, matthews_corrcoef, confusion_matrix

accuracy = accuracy_score(y_test, y_pred_classes)
print("Accuracy:", accuracy)

precision = precision_score(y_test, y_pred_classes, average='weighted')
print("Precision:", precision)

f1 = f1_score(y_test, y_pred_classes, average='weighted')
print("F1 Score:", f1)

mcc = matthews_corrcoef(y_test, y_pred_classes)
print("MCC Score:", mcc)

confusion_mat = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:")
print(confusion_mat)
